# Where in the code library are option definitions hidden?

The following presents a few utility functions the can be used to investigate inheritance and propagation of parameters from class to class.

In [ ]:
import holoviews as hv
import holoviews # for convenience, makes copy-pasting printed class names easier

hv.extension('matplotlib') # only load one backend at a time

# some helper funcs
def all_subclasses(cls):
    """
    All classes that inherit from a given class
    """
    return set(cls.__subclasses__()).union([s for c in cls.__subclasses__() for s in all_subclasses(c)])

def attr_origin(cls, attr):
    """
    return first class in mro that has attribute actually defined as opposed to inherited
    i.e. class where the attribute was originally inherited from
    
    if it returns (object, None) the attr was not present at all
    """
    try:
        for c in cls.mro():
            if attr in c.__dict__:
                break
    finally:
        try:
            val = getattr(c.param, attr)
        except:
            val = getattr(c, attr, None)
        return c, val

def unique_keep_order(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]

from functools import reduce

def common_elements(seqs):
    """
    Get a list of common elements in a seq of seqs, preserving order
    """
    return [i for i in unique_keep_order(reduce(lambda x, y: list(x)+list(y), seqs)) if all(i in l for l in seqs)]

def common_ancestor(clss):
    """
    For a seq of classes, get first root node in mros
    """
    mros = [c.mro() for c in clss]
    return common_elements(mros)[0]

In [ ]:
%pprint 
%pprint

An example:

Pretty much the mother of all plotting classes: `GenericElementPlot`

In [ ]:
# all mpl plotting classes
all_plots = all_subclasses(hv.plotting.plot.GenericElementPlot)
# reality check:
print(common_ancestor(all_plots))

Choose an attribute

In [ ]:
attr = 'show_legend'

In [ ]:
# all plots that define a given attr
attr_origins = set(attr_origin(c, attr) for c in all_plots)
# if attr_origin returns object that means it was not present
plots, params = zip(*((plot, param) for plot, param in attr_origins if plot is not object))
plots

In [ ]:
# inheritance of attr ultimately *could* rely only on these plots, the rest are strictly speaking superfluous
# (assuming leaving unchanged the current inheritance structure)
# i.e. plots from which the other plots inherit the attr (even though they define it themselves)
# i.e. the last in the plot's mro which is also element of plots
strict_parents = set(common_elements([p.mro(), plots])[-1] for p in plots)
strict_parents

In [ ]:
common_ancestor(strict_parents)